## Katie Kimberling - Preprocessing of Liar Dataset

# Preprocessing the Data

In [1]:
import boto3
import os

# Define S3 bucket and file names
s3_bucket = "fake-news-raw-data"
s3_files = ["Liar.csv", "Synthetic Financial Datasets.csv", "WELFake_Dataset.csv"]
local_folder = "/home/ec2-user/SageMaker/data/"

# Ensure local directory exists
os.makedirs(local_folder, exist_ok=True)

# Initialize S3 client
s3_client = boto3.client("s3")

# Download files from S3
for file in s3_files:
    local_path = os.path.join(local_folder, file)
    s3_client.download_file(s3_bucket, file, local_path)
    print(f"✅ Downloaded {file} to {local_path}")

✅ Downloaded Liar.csv to /home/ec2-user/SageMaker/data/Liar.csv
✅ Downloaded Synthetic Financial Datasets.csv to /home/ec2-user/SageMaker/data/Synthetic Financial Datasets.csv
✅ Downloaded WELFake_Dataset.csv to /home/ec2-user/SageMaker/data/WELFake_Dataset.csv


## Import Python package "stopwords" to overlook commonly used words and articles (English)

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Import required modules

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

## Ensure NLTK stopwords are available

In [4]:
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load Liar.csv dataset and make pandas dataframe

In [5]:
liar_clean = pd.read_csv("/home/ec2-user/SageMaker/data/Liar.csv")

# Display dataset info

liar_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Statement ID          10240 non-null  object 
 1   Lie_label             10240 non-null  object 
 2   Statement             10240 non-null  object 
 3   Topic                 10238 non-null  object 
 4   Speaker               10238 non-null  object 
 5   Speaker_Job_Title     7342 non-null   object 
 6   State                 8030 non-null   object 
 7   Speaker_party         10238 non-null  object 
 8   barely_true_counts    10238 non-null  float64
 9   false_counts          10238 non-null  float64
 10  half-true_counts      10238 non-null  float64
 11  mostly_true_counts    10238 non-null  float64
 12  pants_on_fire_counts  10238 non-null  float64
 13  statement_mode        10138 non-null  object 
dtypes: float64(5), object(9)
memory usage: 1.1+ MB


## Function to clean text using NLTK stopwords

In [6]:
def clean_text_nltk(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

liar_clean["clean_statement"] = liar_clean["Statement"].apply(clean_text_nltk)

## Convert numerical columns to integers

In [7]:
count_cols = ["barely_true_counts", "false_counts", "half-true_counts", "mostly_true_counts", "pants_on_fire_counts"]
liar_clean[count_cols] = liar_clean[count_cols].fillna(0).astype(int)

## Create a total misinformation score feature

In [8]:
liar_clean["total_misinfo_score"] = liar_clean[count_cols].sum(axis=1)

## One-Hot Encoding for Speaker Party

In [9]:
encoder = OneHotEncoder(sparse_output=False,drop="first")
encoded_party = encoder.fit_transform(liar_clean[["Speaker_party"]])
party_columns = encoder.get_feature_names_out(["Speaker_party"])
liar_encoded_party = pd.DataFrame(encoded_party, columns=party_columns, index=liar_clean.index)

## Merge and drop original categorical column

In [10]:
liar_clean = pd.concat([liar_clean, liar_encoded_party], axis=1)
liar_clean.drop(["Speaker_party"], axis=1, inplace=True)

## Balance dataset by oversampling minority classes

In [11]:
majority_class = liar_clean[liar_clean["Lie_label"] == "FALSE"]
minority_classes = liar_clean[liar_clean["Lie_label"] != "FALSE"]
minority_classes_upsampled = resample(minority_classes, replace=True, n_samples=len(majority_class), random_state=42)
liar_balanced = pd.concat([majority_class, minority_classes_upsampled])

## Split dataset

In [12]:
train_data, test_data = train_test_split(liar_balanced, test_size=0.2, random_state=42, stratify=liar_balanced["Lie_label"])

## Save the preprocessed data

In [13]:
train_data.to_csv("/home/ec2-user/SageMaker/data/Liar_train.csv", index=False)
test_data.to_csv("/home/ec2-user/SageMaker/data/Liar_test.csv", index=False)

print("Preprocessing on Liar dataset complete. Training and test datasets saved.")

Preprocessing on Liar dataset complete. Training and test datasets saved.


**References**

OpenAI. (2025). ChatGPT (March 20 version). [LLM]. https://chatgpt.com

Python Tutorials. (2021, July 22). *NLTK stop words.* pythonspot. Accessed March 20, 2025 from https://pythonspot.com/nltk-stop-words/

## Katie Kimberling's Data Training Liar Dataset

## Import necessary packages

In [14]:
import pandas as pd
import boto3
import sagemaker
from sagemaker import get_execution_role

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Reload the train/test datasets

In [15]:
train_data = pd.read_csv("/home/ec2-user/SageMaker/data/Liar_train.csv")
test_data = pd.read_csv("/home/ec2-user/SageMaker/data/Liar_test.csv")

## Define BlazingText format function and output paths

In [16]:
def prepare_blazingtext_format(df, text_col, label_col, output_file):
    with open(output_file, 'w') as f:
        for _, row in df.iterrows():
            label = f"__label__{row[label_col]}"
            text = row[text_col]
            f.write(f"{label} {text}\n")

# Paths to save formatted training and test data
train_txt_path = "/home/ec2-user/SageMaker/data/liar_train_blazing.txt"
test_txt_path = "/home/ec2-user/SageMaker/data/liar_test_blazing.txt"

## Format data for BlazingText

In [17]:
prepare_blazingtext_format(train_data, text_col="clean_statement", label_col="Lie_label", output_file=train_txt_path)
prepare_blazingtext_format(test_data, text_col="clean_statement", label_col="Lie_label", output_file=test_txt_path)

print("Training and test data formatted for BlazingText.")

Training and test data formatted for BlazingText.


## Upload formatted data to s3

In [18]:
import sagemaker
sess = sagemaker.Session()

# S3 upload paths
s3_bucket = sess.default_bucket()
s3_prefix = 'blazingtext-data'

train_s3_path = f'{s3_prefix}/liar_train_blazing.txt'
test_s3_path = f'{s3_prefix}/liar_test_blazing.txt'

# Upload to S3
s3 = boto3.client('s3')
s3.upload_file(train_txt_path, s3_bucket, train_s3_path)
s3.upload_file(test_txt_path, s3_bucket, test_s3_path)

print("Files uploaded to S3.")

Files uploaded to S3.


## Set up role, session and input paths

In [19]:
# Role and session
role = get_execution_role()
sess = sagemaker.Session()

# S3 input paths
s3_train_input = f's3://{s3_bucket}/{train_s3_path}'
s3_test_input = f's3://{s3_bucket}/{test_s3_path}'

## Define BlazingText estimator

In [20]:
from sagemaker.amazon.amazon_estimator import get_image_uri

bt_image = sagemaker.image_uris.retrieve("blazingtext", sess.boto_region_name)
bucket = sess.default_bucket()
bt_estimator = sagemaker.estimator.Estimator(
    image_uri=bt_image,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size=5,
    max_run=3600,
    input_mode='File',
    output_path=f's3://{bucket}/blazingtext-output',
    sagemaker_session=sess
)

[04/04/25 00:49:12] INFO     Same images used for training and inference. Defaulting to image     ]8;id=780640;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=690677;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#393\393]8;;\
                             scope: inference.                                                                     

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=667127;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=225179;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

# Set hyperparameters

In [21]:
bt_estimator.set_hyperparameters(
    mode='supervised',
    epochs=10,
    learning_rate=0.05,
    vector_dim=100,
    min_count=2,
    early_stopping=True
)

## Launch training and deploy model

In [22]:
import json
import boto3
import sagemaker
from sagemaker.inputs import TrainingInput

train_input = TrainingInput(s3_data=s3_train_input, content_type='text/plain')
test_input = TrainingInput(s3_data=s3_test_input, content_type='text/plain')
bt_estimator.fit({'train': train_input, 'validation': test_input})



[04/04/25 00:49:13] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=372541;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=908624;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: blazingtext-2025-04-04-00-49-13-104   ]8;id=418205;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=443630;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-04-04 00:49:14 Starting - Starting the training job...
..25-04-04 00:49:48 Downloading - Downloading input data.
..25-04-04 00:50:09 Downloading - Downloading the training image.
2025-04-04 00:50:24 Training - Training image download completed. Training in progress.Arguments: train
[04/04/2025 00:50:37 WARNING 139961220372288] Loggers have already been setup.
[04/04/2025 00:50:37 WARNING 139961220372288] Loggers have already been setup.
/opt/amazon/python3.8/lib/python3.8/subprocess.py:848: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
[04/04/2025 00:50:37 INFO 139961220372288] nvidia-smi took: 0.025159597396850586 secs to identify 0 gpus
[04/04/2025 00:50:37 INFO 139961220372288] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
[04/04/2025 00:50:37 INFO 139961220372288] Processing /opt/ml/input/data

In [23]:
# Deploy the model
predictor = bt_estimator.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

[04/04/25 00:52:00] INFO     Creating model with name: blazingtext-2025-04-04-00-52-00-882          ]8;id=471999;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=28119;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Deploy the model                                                                           │
│ ❱ 2 predictor = bt_estimator.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')      │
│   3                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/estimator.py:1722 in deploy                    │
│                                                                                                  │
│   1719 │   │   │   inference_tags=format_tags(tags), training_tags=self.tags                     │
│   1720 │   │   )                                                                                 │
│   1721 │   │                                                                                     │
│ ❱ 1722 │   │   return model.deploy(                                                              │
│   1723 │   │   │   instance_type=instance_type,                                                  │
│   1724 │   │   │   initial_instance_count=initial_instance_count,                                │
│   1725 │   │   │   serializer=serializer,                                                        │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model.py:1728 in deploy                        │
│                                                                                                  │
│   1725 │   │   │   return None                                                                   │
│   1726 │   │                                                                                     │
│   1727 │   │   else:  # existing single model endpoint path                                      │
│ ❱ 1728 │   │   │   self._create_sagemaker_model(                                                 │
│   1729 │   │   │   │   instance_type=instance_type,                                              │
│   1730 │   │   │   │   accelerator_type=accelerator_type,                                        │
│   1731 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model.py:985 in _create_sagemaker_model        │
│                                                                                                  │
│    982 │   │   │   │   enable_network_isolation=self._enable_network_isolation,                  │
│    983 │   │   │   │   tags=format_tags(tags),                                                   │
│    984 │   │   │   )                                                                             │
│ ❱  985 │   │   │   self.sagemaker_session.create_model(**create_model_args)                      │
│    986 │                                                                                         │
│    987 │   def _get_model_uri(self):                                                             │
│    988 │   │   model_uri = None                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/session.py:4109 in create_model                │
│                                                                                                  │
│   4106 │   │   │   │   else:                               

## Evaluate the model

In [ ]:
from sklearn.metrics import classification_report
import json
import boto3
import sagemaker
import json

# Prepare test input
test_statements = test_data['clean_statement'].tolist()
payload = {"instances": test_statements}

# Get Response
runtime = boto3.client("sagemaker-runtime")
response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType="application/json",  # Ensure JSON content type
    Body=json.dumps(payload).encode("utf-8")  # Encode the JSON payload
)

# Actual true labels
true_labels = test_data['Lie_label'].tolist()

# Parse response
result = json.loads(response["Body"].read().decode())
predicted_labels = ["TRUE" if "TRUE" in p["label"][0] else "FALSE" for p in result]

# Evaluate
print(classification_report(true_labels, predicted_labels))

## Cleanup the endpoint, or else goodbye, $$

In [ ]:
# Cleanup
predictor.delete_endpoint()